## Introducción

En el presente trabajo se propone realizar un análisis de los datos recolectados por una empresa en los últimos 2 meses sobre los clientes que fueron a ver la pelicula Frozen 3. La encuesta consistía en una serie de preguntas personales que concluía si el cliente volvería a ese cine si salía la secuela, Frozen 4.

El objetivo de la empresa es poder dirigir futuras campañas de marketing digital en base a esta información recolectada.

## Objetivos

El objetivo de este notebook es:
*   Entender los datos
*   Ver cómo los datos se relacionan entre si
*   Poder sacar alguna conclusión o descubrir un patrón a partir de estos
*   Indicar cuáles son los factores más importantes que determinan si un usuario va a ir al cine a ver Frozen 4 o no.
*   Poder llegar a armar un baseline

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from math import pi

## Carga y limpieza del set de datos

In [65]:
usuario_volveria_df = pd.read_csv(r'C:\Users\Enrique\Desktop\analisis\Datasets\tp-2020-2c-train-cols1.csv')
usuario_volveria_df.head()

,id_usuario,volveria
0,117,0
1,658,0
2,794,0
3,455,0
4,173,1


In [66]:
info_fiumark_df = pd.read_csv(r'C:\Users\Enrique\Desktop\analisis\Datasets\tp-2020-2c-train-cols2.csv')
info_fiumark_df.head()

,tipo_de_sala,nombre,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede
0,4d,Señor Camilo Pedro,117,hombre,73.5,0,0,59258;,1,NaN,fiumark_quilmes
1,4d,Señora Raquel Angelica,658,mujer,35.0,1,1,586:6;,2,NaN,fiumark_quilmes
2,normal,Señor Antonio Federico,794,hombre,NaN,0,0,"RE""39822",3,NaN,fiumark_chacarita
3,4d,Señor Osvaldo Aureliano,455,hombre,NaN,0,0,"C17""4:39",1,NaN,fiumark_palermo
4,4d,Señorita Rita Eudosia,173,mujer,4.0,1,1,569964,2,NaN,fiumark_palermo


### df volveria

In [67]:
usuario_volveria_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   id_usuario  801 non-null    int64
 1   volveria    801 non-null    int64
dtypes: int64(2)
memory usage: 12.6 KB


Notamos que la columna de 'volveria' es de tipo int64, cuando solamente tiene valores que pueden ser 0 o 1. Un mejor tipo de dato para este caso puede ser un int8. Esto permitiría ahorrar memoria. (Aunque en este caso no sean muchos datos)

In [68]:
usuario_volveria_df['volveria'] = usuario_volveria_df['volveria'].astype(np.int8)

Observamos que no tenemos valores nulos

In [70]:
usuario_volveria_df.isnull().sum()

id_usuario    0
volveria      0
dtype: int64

### info_fiumark_df

Notamos una gran cantidad de valores nulos en la columna "fila" y algunos en "edad" y "nombre_sede". Como las columnas que tienen los valores nulos pueden llegar a ser relevantes para el análisis, siendo este el caso de las columnas "fila" y "edad" decidimos crear la categoría "No responde" para el caso de "fila".

In [71]:
info_fiumark_df.isnull().sum()

tipo_de_sala       0
nombre             0
id_usuario         0
genero             0
edad             160
amigos             0
parientes          0
id_ticket          0
precio_ticket      0
fila             624
nombre_sede        2
dtype: int64

In [72]:
info_fiumark_df['fila'].fillna("No responde",inplace=True)

Ahora vemos la cantidad de valores únicos para las columnas de tipo objeto, para poder determinar a cuales les correspondería tener un tipo categórico.

In [73]:
info_fiumark_df.select_dtypes(include=['object']).nunique()

tipo_de_sala      3
nombre          801
genero            2
id_ticket       635
fila              3
nombre_sede       3
dtype: int64

In [74]:
info_fiumark_df["tipo_de_sala"] = info_fiumark_df["tipo_de_sala"].astype("category")
info_fiumark_df["genero"] = info_fiumark_df["genero"].astype("category")
info_fiumark_df["nombre_sede"] = info_fiumark_df["nombre_sede"].astype("category")
info_fiumark_df["fila"] = info_fiumark_df["fila"].astype("category")

Otra cosa que observamos es que las edades están en formato de float, lo cual nos resultó llamativo, por lo que decidimos cortar los valores, llevándolos al piso.

In [75]:
info_fiumark_df['edad'].value_counts()

27.00    28
22.00    25
25.00    25
31.00    23
21.00    23
         ..
77.00     1
27.50     1
3.67      1
58.50     1
26.50     1
Name: edad, Length: 84, dtype: int64

In [76]:
info_fiumark_df['edad'] = info_fiumark_df['edad'].apply(np.floor)

In [77]:
info_fiumark_df['edad'].value_counts()

27.0    29
22.0    25
25.0    25
31.0    24
33.0    24
        ..
62.0     1
83.0     1
15.0     1
69.0     1
66.0     1
Name: edad, Length: 71, dtype: int64

Pudimos observar que una parte en la columna de 'edad' tiene valores faltantes. Lo que decidimos hacer fue utilizar la columna 'nombre', que incluye sufijos como 'Señora' o 'Señorita' para rellenar el valor de edad.

Lo primero que hacemos es crear una nueva columna 'sufijo' a partir de los valores de nombres anteriores.

In [78]:
sufijos_y_nombres = info_fiumark_df.nombre.str.split(pat = ' ', n= 1,expand = True)
sufijos_y_nombres.columns = ['sufijo', 'nombre']

In [79]:
info_fiumark_df.drop('nombre', axis = 1, inplace = True)

In [80]:
info_fiumark_df = pd.concat([ sufijos_y_nombres, info_fiumark_df ], axis = 'columns')

In [82]:
info_fiumark_df.head()

,sufijo,nombre,tipo_de_sala,id_usuario,genero,edad,amigos,parientes,id_ticket,precio_ticket,fila,nombre_sede
0,Señor,Camilo Pedro,4d,117,hombre,73.0,0,0,59258;,1,No responde,fiumark_quilmes
1,Señora,Raquel Angelica,4d,658,mujer,35.0,1,1,586:6;,2,No responde,fiumark_quilmes
2,Señor,Antonio Federico,normal,794,hombre,NaN,0,0,"RE""39822",3,No responde,fiumark_chacarita
3,Señor,Osvaldo Aureliano,4d,455,hombre,NaN,0,0,"C17""4:39",1,No responde,fiumark_palermo
4,Señorita,Rita Eudosia,4d,173,mujer,4.0,1,1,569964,2,No responde,fiumark_palermo


Ahora reemplazamos la edad de las 'Señoras' que no contestaron la edad por la mediana de las que sí contestaron. Hacemos lo mismo para las 'Señoritas'.

Ademas creamos una columna 'autocompletamos_edad' para indicar cuáles de estas edades fueron completadas por nosotros.

In [83]:
mediana_edad_senioras = info_fiumark_df[info_fiumark_df['sufijo'] == 'Señora'].edad.dropna().median()
mediana_edad_senioritas = info_fiumark_df[info_fiumark_df['sufijo'] == 'Señorita'].edad.dropna().median()

In [84]:
info_fiumark_df['autocompletamos_edad'] = False

In [86]:
info_fiumark_df.loc[(info_fiumark_df['sufijo'] == 'Señora') & (info_fiumark_df['edad'].isnull()),'autocompletamos_edad'] = True
info_fiumark_df.loc[(info_fiumark_df['sufijo'] == 'Señorita') & (info_fiumark_df['edad'].isnull()),'autocompletamos_edad'] = True

In [90]:
info_fiumark_df['autocompletamos_edad'].value_counts()

False    755
True      46
Name: autocompletamos_edad, dtype: int64

In [93]:
info_fiumark_df.loc[info_fiumark_df['sufijo'] == 'Señorita','edad'] = info_fiumark_df.loc[info_fiumark_df['sufijo'] == 'Señorita','edad'].fillna(mediana_edad_senioritas)
info_fiumark_df.loc[info_fiumark_df['sufijo'] == 'Señora','edad'] = info_fiumark_df.loc[info_fiumark_df['sufijo'] == 'Señora','edad'].fillna(mediana_edad_senioras) 

Veamos ahora el caso de los hombres. Algo que notamos en este caso, es que no tenemos el sufijo 'Señorito' como para tener una distinción muy marcada en comparación a la de las mujeres. Pero observando que la cantidad de niños que respondieron la encuesta es mucho menor en proporción a la de los hombres, decidimos realizar el reemplazo tomando también la mediana.

In [94]:
def cantidad_total_de_ninios(edad):
    ninios = edad<=10
    return ninios.sum()

info_fiumark_df.groupby(by='genero').agg({'edad':cantidad_total_de_ninios}).rename(columns={'edad':'Cantidad de ninios que respondieron la encuesta'})

In [97]:
mediana_edad_senior = info_fiumark_df[info_fiumark_df['sufijo'] == 'Señor'].edad.dropna().median()

In [98]:
info_fiumark_df.loc[(info_fiumark_df['sufijo'] == 'Señor') & (info_fiumark_df['edad'].isnull()),'autocompletamos_edad'] = True
info_fiumark_df.loc[info_fiumark_df['sufijo'] == 'Señor','edad'] = info_fiumark_df.loc[info_fiumark_df['sufijo'] == 'Señor','edad'].fillna(mediana_edad_senior)

También observamos que entre los datos que nos fueron entregados se encuentra el código de ticket: "id_ticket". Consideramos que la información en ella es irrelevante para nuestros propósitos y por lo tanto la eliminamos. Esta decisión se debe a que parece que el código del ticket tiene forma de ser generada de forma aleatoria y no da la impresión de estar relacionada con la variable de interés (si volvería) para el análisis.

In [100]:
info_fiumark_df.drop(columns='id_ticket',inplace=True)

Ahora revisamos que la columna nombre tenga efectivamente nombres. Por ejemplo, esta columna no puede tener numeros.

In [102]:
info_fiumark_df['nombre'].str.contains('1|2|3|4|5|6|7|8|9|0|#|%|&|!|@').any()

False

Vemos que la columna 'nombre' esta compuesta por caracteres validos.

Habiendo hecho esos cambios, finalmente nos queda el dataframe de la siguiente forma. En el proceso se redujo el uso de memoria, pasando de 69.0KB a 48.2KB.

In [106]:
info_fiumark_df.head()

,sufijo,nombre,tipo_de_sala,id_usuario,genero,edad,amigos,parientes,precio_ticket,fila,nombre_sede,autocompletamos_edad
0,Señor,Camilo Pedro,4d,117,hombre,73.0,0,0,1,No responde,fiumark_quilmes,False
1,Señora,Raquel Angelica,4d,658,mujer,35.0,1,1,2,No responde,fiumark_quilmes,False
2,Señor,Antonio Federico,normal,794,hombre,32.0,0,0,3,No responde,fiumark_chacarita,True
3,Señor,Osvaldo Aureliano,4d,455,hombre,32.0,0,0,1,No responde,fiumark_palermo,True
4,Señorita,Rita Eudosia,4d,173,mujer,4.0,1,1,2,No responde,fiumark_palermo,False


Por último, sacamos la columna de 'id_usuario', ya que no es necesaria para el análisis.

In [107]:
info_fiumark_df.drop(columns='id_usuario',inplace=True)
info_fiumark_df.head()

,sufijo,nombre,tipo_de_sala,genero,edad,amigos,parientes,precio_ticket,fila,nombre_sede,autocompletamos_edad
0,Señor,Camilo Pedro,4d,hombre,73.0,0,0,1,No responde,fiumark_quilmes,False
1,Señora,Raquel Angelica,4d,mujer,35.0,1,1,2,No responde,fiumark_quilmes,False
2,Señor,Antonio Federico,normal,hombre,32.0,0,0,3,No responde,fiumark_chacarita,True
3,Señor,Osvaldo Aureliano,4d,hombre,32.0,0,0,1,No responde,fiumark_palermo,True
4,Señorita,Rita Eudosia,4d,mujer,4.0,1,1,2,No responde,fiumark_palermo,False
